In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8731,2021-04-14T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1230,185,1415,22717,...,NaN,15.0,NaN,NaN,190523.0,0.0,2131613.0,1341503.0,ITG,ITG1
8732,2021-04-14T17:00:00,ITA,9,Toscana,43.769231,11.255889,1664,282,1946,25131,...,NaN,17.0,NaN,NaN,209554.0,2471.0,3110136.0,640989.0,ITI,ITI1
8733,2021-04-14T17:00:00,ITA,10,Umbria,43.106758,12.388247,262,38,300,3488,...,"Si fa presente che 7 dei ricoveri NON UTI, non...",1.0,NaN,NaN,52730.0,0.0,820599.0,239763.0,ITI,ITI2
8734,2021-04-14T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,66,12,78,1064,...,NaN,2.0,NaN,NaN,9826.0,369.0,85384.0,19636.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8731,2021-04-14T17:00:00,19,Sicilia,1230,185,1415,22717,24132,-539,1542,...,190523,3473116,1361557.0,15.0,190523.0,0.0,2131613.0,1341503.0,ITG,ITG1
8732,2021-04-14T17:00:00,9,Toscana,1664,282,1946,25131,27077,-337,1168,...,212025,3751125,1882641.0,17.0,209554.0,2471.0,3110136.0,640989.0,ITI,ITI1
8733,2021-04-14T17:00:00,10,Umbria,262,38,300,3488,3788,-68,137,...,52730,1060362,346925.0,1.0,52730.0,0.0,820599.0,239763.0,ITI,ITI2
8734,2021-04-14T17:00:00,2,Valle d'Aosta,66,12,78,1064,1142,22,60,...,10195,105020,55066.0,2.0,9826.0,369.0,85384.0,19636.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-04-14', '2021-04-13')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-04-14T17:00:00,1,Piemonte,3430,311,3741,21517,25258,-1131,1439,...,329708,3626825,1602089.0,11.0,313672.0,16036.0,2531232.0,1095593.0,ITC,ITC1
1,2021-04-14T17:00:00,2,Valle d'Aosta,66,12,78,1064,1142,22,60,...,10195,105020,55066.0,2.0,9826.0,369.0,85384.0,19636.0,ITC,ITC2
2,2021-04-14T17:00:00,3,Lombardia,5589,781,6370,63694,70064,-1277,2153,...,772038,8773329,3627106.0,30.0,731704.0,40334.0,7699523.0,1073806.0,ITC,ITC4
3,2021-04-14T17:00:00,5,Veneto,1461,260,1721,27721,29442,-1117,1081,...,397758,6628164,1612704.0,10.0,387796.0,9962.0,4766501.0,1861663.0,ITH,ITH3
4,2021-04-14T17:00:00,6,Friuli Venezia Giulia,471,72,543,9783,10326,-203,226,...,102088,1727870,611041.0,6.0,88413.0,13675.0,1510510.0,217360.0,ITH,ITH4
5,2021-04-14T17:00:00,7,Liguria,633,84,717,6477,7194,-271,410,...,94879,1332679,551608.0,6.0,94879.0,0.0,1115912.0,216767.0,ITC,ITC3
6,2021-04-14T17:00:00,8,Emilia-Romagna,2590,317,2907,62210,65117,164,859,...,353761,5183846,1722577.0,14.0,353512.0,249.0,4179419.0,1004427.0,ITH,ITH5
7,2021-04-14T17:00:00,9,Toscana,1664,282,1946,25131,27077,-337,1168,...,212025,3751125,1882641.0,17.0,209554.0,2471.0,3110136.0,640989.0,ITI,ITI1
8,2021-04-14T17:00:00,10,Umbria,262,38,300,3488,3788,-68,137,...,52730,1060362,346925.0,1.0,52730.0,0.0,820599.0,239763.0,ITI,ITI2
9,2021-04-14T17:00:00,11,Marche,671,116,787,7069,7856,-94,415,...,93230,1065926,641913.0,5.0,93230.0,0.0,957171.0,108755.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-04-13T17:00:00,1,Piemonte,3526,316,3842,22547,26389,-1272,1057,...,328269,3604839,1595062.0,21.0,312347.0,15922.0,2518699.0,1086140.0,ITC,ITC1
1,2021-04-13T17:00:00,2,Valle d'Aosta,68,12,80,1040,1120,-9,48,...,10135,104333,54917.0,1.0,9775.0,360.0,84941.0,19392.0,ITC,ITC2
2,2021-04-13T17:00:00,3,Lombardia,5727,787,6514,64827,71341,-1346,1975,...,769885,8722842,3618520.0,58.0,729977.0,39908.0,7667284.0,1055558.0,ITC,ITC4
3,2021-04-13T17:00:00,5,Veneto,1534,269,1803,28756,30559,-1128,883,...,396677,6587690,1608301.0,18.0,386762.0,9915.0,4747307.0,1840383.0,ITH,ITH3
4,2021-04-13T17:00:00,6,Friuli Venezia Giulia,493,75,568,9961,10529,-458,371,...,101862,1720725,609751.0,3.0,88238.0,13624.0,1504236.0,216489.0,ITH,ITH4
5,2021-04-13T17:00:00,7,Liguria,658,87,745,6720,7465,-268,231,...,94469,1324219,548841.0,7.0,94469.0,0.0,1110318.0,213901.0,ITC,ITC3
6,2021-04-13T17:00:00,8,Emilia-Romagna,2654,331,2985,61968,64953,-1480,785,...,352908,5156399,1718783.0,19.0,352659.0,249.0,4163612.0,992787.0,ITH,ITH5
7,2021-04-13T17:00:00,9,Toscana,1664,283,1947,25467,27414,-217,934,...,210857,3725470,1874315.0,13.0,208421.0,2436.0,3094299.0,631171.0,ITI,ITI1
8,2021-04-13T17:00:00,10,Umbria,279,41,320,3536,3856,-46,109,...,52593,1052934,346247.0,2.0,52593.0,0.0,817825.0,235109.0,ITI,ITI2
9,2021-04-13T17:00:00,11,Marche,713,127,840,7110,7950,-105,222,...,92815,1060551,639222.0,10.0,92815.0,0.0,952504.0,108047.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-04-14T17:00:00,1,Piemonte,3430,311,3741,21517,25258,-1131,1439,...,3626825,1602089.0,11.0,313672.0,16036.0,2531232.0,1095593.0,ITC,ITC1,4.341375e+06
1,2021-04-14T17:00:00,2,Valle d'Aosta,66,12,78,1064,1142,22,60,...,105020,55066.0,2.0,9826.0,369.0,85384.0,19636.0,ITC,ITC2,1.255010e+05
2,2021-04-14T17:00:00,3,Lombardia,5589,781,6370,63694,70064,-1277,2153,...,8773329,3627106.0,30.0,731704.0,40334.0,7699523.0,1073806.0,ITC,ITC4,1.010397e+07
3,2021-04-14T17:00:00,5,Veneto,1461,260,1721,27721,29442,-1117,1081,...,6628164,1612704.0,10.0,387796.0,9962.0,4766501.0,1861663.0,ITH,ITH3,4.907704e+06
4,2021-04-14T17:00:00,6,Friuli Venezia Giulia,471,72,543,9783,10326,-203,226,...,1727870,611041.0,6.0,88413.0,13675.0,1510510.0,217360.0,ITH,ITH4,1.211357e+06
5,2021-04-14T17:00:00,7,Liguria,633,84,717,6477,7194,-271,410,...,1332679,551608.0,6.0,94879.0,0.0,1115912.0,216767.0,ITC,ITC3,1.543127e+06
6,2021-04-14T17:00:00,8,Emilia-Romagna,2590,317,2907,62210,65117,164,859,...,5183846,1722577.0,14.0,353512.0,249.0,4179419.0,1004427.0,ITH,ITH5,4.467118e+06
7,2021-04-14T17:00:00,9,Toscana,1664,282,1946,25131,27077,-337,1168,...,3751125,1882641.0,17.0,209554.0,2471.0,3110136.0,640989.0,ITI,ITI1,3.722729e+06
8,2021-04-14T17:00:00,10,Umbria,262,38,300,3488,3788,-68,137,...,1060362,346925.0,1.0,52730.0,0.0,820599.0,239763.0,ITI,ITI2,8.802850e+05
9,2021-04-14T17:00:00,11,Marche,671,116,787,7069,7856,-94,415,...,1065926,641913.0,5.0,93230.0,0.0,957171.0,108755.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-04-14T17:00:00,1,Piemonte,3430,311,3741,21517,25258,-1131,1439,...,1095593.0,ITC,ITC1,4.341375e+06,Piemonte,3430,5824,311,628,99
1,2021-04-14T17:00:00,2,Valle d'Aosta,66,12,78,1064,1142,22,60,...,19636.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,66,227,12,30,6
2,2021-04-14T17:00:00,3,Lombardia,5589,781,6370,63694,70064,-1277,2153,...,1073806.0,ITC,ITC4,1.010397e+07,Lombardia,5589,12998,781,1416,114
3,2021-04-14T17:00:00,5,Veneto,1461,260,1721,27721,29442,-1117,1081,...,1861663.0,ITH,ITH3,4.907704e+06,Veneto,1461,6000,260,1000,0
4,2021-04-14T17:00:00,6,Friuli Venezia Giulia,471,72,543,9783,10326,-203,226,...,217360.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,471,1277,72,175,0
5,2021-04-14T17:00:00,7,Liguria,633,84,717,6477,7194,-271,410,...,216767.0,ITC,ITC3,1.543127e+06,Liguria,633,1854,84,226,12
6,2021-04-14T17:00:00,8,Emilia-Romagna,2590,317,2907,62210,65117,164,859,...,1004427.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,2590,6538,317,760,0
7,2021-04-14T17:00:00,9,Toscana,1664,282,1946,25131,27077,-337,1168,...,640989.0,ITI,ITI1,3.722729e+06,Toscana,1664,5033,282,632,13
8,2021-04-14T17:00:00,10,Umbria,262,38,300,3488,3788,-68,137,...,239763.0,ITI,ITI2,8.802850e+05,Umbria,262,844,38,132,2
9,2021-04-14T17:00:00,11,Marche,671,116,787,7069,7856,-94,415,...,108755.0,ITI,ITI3,1.518400e+06,Marche,671,1410,116,247,19


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-04-14T17:00:00,15,Campania,1585,127,1712,89796,91508,570,2212,...,20.48,583,585,24938,15423.0,27,1615,8.87,0.03823,6.26
1,2021-04-14T17:00:00,3,Lombardia,5589,781,6370,63694,70064,-1277,2153,...,55.16,-1133,178,50487,8586.0,85,3345,4.26,0.02131,7.64
2,2021-04-14T17:00:00,19,Sicilia,1230,185,1415,22717,24132,-539,1542,...,22.16,-564,158,29503,11579.0,33,2048,5.23,0.03104,3.83
3,2021-04-14T17:00:00,16,Puglia,1935,270,2205,49584,51789,155,1488,...,46.08,178,297,13647,2359.0,39,1294,10.90,0.03712,5.34
4,2021-04-14T17:00:00,1,Piemonte,3430,311,3741,21517,25258,-1131,1439,...,49.52,-1030,382,21986,7027.0,70,2500,6.55,0.03315,7.59
5,2021-04-14T17:00:00,12,Lazio,2977,398,3375,47857,51232,-940,1230,...,42.21,-859,66,38967,20231.0,49,2121,3.16,0.02097,5.20
6,2021-04-14T17:00:00,9,Toscana,1664,282,1946,25131,27077,-337,1168,...,44.62,-336,234,25655,8326.0,32,1473,4.55,0.03137,5.70
7,2021-04-14T17:00:00,5,Veneto,1461,260,1721,27721,29442,-1117,1081,...,26.00,-1035,198,40474,4403.0,24,2174,2.67,0.02203,8.10
8,2021-04-14T17:00:00,8,Emilia-Romagna,2590,317,2907,62210,65117,164,859,...,41.71,242,74,27447,3794.0,30,659,3.13,0.01923,7.92
9,2021-04-14T17:00:00,18,Calabria,476,44,520,12504,13024,117,540,...,28.95,111,-37,3785,3426.0,6,417,14.27,0.02806,2.75


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,781,635,1416,55.16
1,Piemonte,311,317,628,49.52
2,Marche,116,131,247,46.96
3,Puglia,270,316,586,46.08
4,Toscana,282,350,632,44.62
5,Molise,17,22,39,43.59
6,Lazio,398,545,943,42.21
7,Emilia-Romagna,317,443,760,41.71
8,Friuli Venezia Giulia,72,103,175,41.14
9,P.A. Trento,37,53,90,41.11


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))